In [1]:
import numpy as np
import pandas as pd
import re
from datetime import datetime, timezone
from scipy.stats import norm

def bs_call_price(S, K, T, r, q, sigma):
    d1 = (np.log(S/K) + (r - q + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*np.exp(-q*T)*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

def implied_vol_call(S, K, T, r, q, price, tol=1e-8, max_iter=100, init=0.2):
    sigma = max(1e-6, init)
    for _ in range(max_iter):
        d1 = (np.log(S/K) + (r - q + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
        d2 = d1 - sigma*np.sqrt(T)
        model = S*np.exp(-q*T)*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
        vega  = S*np.exp(-q*T)*norm.pdf(d1)*np.sqrt(T)
        diff = model - price
        if abs(diff) < tol: 
            return sigma
        if vega < 1e-10:
            break
        sigma -= diff/vega
        sigma = min(max(sigma, 1e-6), 5.0)
    return sigma

In [17]:
import re
import numpy as np
import pandas as pd

path = "data/calls_21_11_25.xlsx"  # adapte

# 1) lire avec les en-têtes (sans skip) pour récupérer le bandeau (1re ligne)
raw = pd.read_excel(path, engine="openpyxl")

# concatène toutes les cellules non vides de la première ligne en une seule chaîne
row0 = raw.iloc[0]
info_text = " ".join(str(x) for x in row0.dropna().astype(str).tolist()).strip()
print("Bandeau lu ->", info_text)

# 2) regex robustes
m_date = re.search(r"(\d{1,2}-[A-Za-z]{3}-\d{2})", info_text)
m_ff   = re.search(r"FF\s*([0-9]*\.?[0-9]+)", info_text)
m_r    = re.search(r"\bR\s*([0-9]*\.?[0-9]+)", info_text)
m_div  = re.search(r"(?:DivI|DIVI)\s*[=:]?\s*([0-9]*\.?[0-9]+)", info_text)

if not m_date:
    # 🔁 fallback: essaie de lire la date depuis le NOM DE FICHIER (ex: puts_16_01_26.xlsx)
    m_fn = re.search(r"(\d{2})_(\d{2})_(\d{2})", path)
    if m_fn:
        d, m, y = m_fn.groups()
        m_date = pd.Timestamp(f"20{y}-{m}-{d}")  # 16_01_26 -> 2026-01-16
    else:
        raise ValueError(f"Expiration introuvable dans le bandeau: {info_text}")
else:
    m_date = pd.to_datetime(m_date.group(1), format="%d-%b-%y")

F = float(m_ff.group(1)) if m_ff else np.nan
r = float(m_r.group(1))/100.0 if m_r else np.nan   # ex "R 4.20" -> 0.042
q = float(m_div.group(1))/100.0 if m_div else np.nan  # ex "DivI .21" -> 0.0021

print(f"Expiration={m_date.date()} | F≈{F} | r≈{r} | q≈{q}")

# 3) relis la table **utile** en sautant UNIQUEMENT la 2e ligne (celle du bandeau)
df = pd.read_excel(path, skiprows=[1], engine="openpyxl")

# nettoyage basique
for c in ["Strike","Bid","Ask","Dern","VIM"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")
df = df.dropna(subset=["Strike"]).reset_index(drop=True)

print(df.head())


Bandeau lu -> 21-Nov-25 (44j); TailleC 100; ; DivI.43; R 4.20; FF 6725.21
Expiration=2025-11-21 | F≈6725.21 | r≈0.042 | q≈0.0043
   Strike              Ticker         Bid         Ask        Dern        VIM  \
0    6595  SPX 11/21/25 C6595  237.199997  238.399994  250.949997  15.213007   
1    6600  SPX 11/21/25 C6600  233.300003  234.600006  229.360001  15.147107   
2    6605  SPX 11/21/25 C6605  229.500000  230.899994  223.050003  15.089969   
3    6610  SPX 11/21/25 C6610  225.600006  226.899994  234.800003  15.007057   
4    6615  SPX 11/21/25 C6615  221.899994  223.100006  227.139999  14.957450   

   Vol.  
0     0  
1     0  
2     0  
3     0  
4     0  


In [18]:
today = pd.Timestamp.today().normalize()
T_days = max((m_date - today).days, 0)
T = T_days / 365.0

print(f"T = {T_days} jours ≈ {T:.3f} ans")

# 0) Assurer Mid & VIM propres
df = df.copy()
for c in ["Strike","Bid","Ask","Dern","VIM"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# Mid = (Bid+Ask)/2 si possible, sinon Dern (Last)
df["Mid"] = np.where(df[["Bid","Ask"]].notna().all(axis=1),
                     (df["Bid"]+df["Ask"])/2.0,
                     df["Dern"])
df = df.dropna(subset=["Strike","Mid"])

# VIM Bloomberg est souvent en %, mais on gère les deux cas (%, décimal)
def _vim_to_pct(v):
    if pd.isna(v): return np.nan
    return v*100 if 0 <= v <= 1 else v
df["VIM_pct"] = df["VIM"].apply(_vim_to_pct)

# 1) Choix de S (spot) – on a généralement F (forward) et r,q,T : S = F*exp(-(r-q)T)
if np.isfinite(F):
    S = float(F*np.exp(-(r - q)*T))
else:
    # fallback simple si F manquant
    S = float(df["Strike"].median())

# 2) Prendre 4 strikes les + proches de l’ATM (S)
df_sorted = df.iloc[(df["Strike"] - S).abs().nsmallest(4).index].sort_values("Strike")

# 3) Recalcul IV (Newton) sur ces 4 calls
rows = []
for _, row in df_sorted.iterrows():
    K = float(row["Strike"])
    price_mkt = float(row["Mid"])
    # point de départ pour Newton: utilise VIM si dispo (en décimal), sinon 20%
    init_sigma = (row["VIM_pct"]/100.0) if pd.notna(row["VIM_pct"]) else 0.20
    iv_calc = implied_vol_call(S, K, T, r, q, price_mkt, init=float(init_sigma))

    rows.append({
        "Strike": K,
        "MidPrice": round(price_mkt, 4),
        "VIM_bbg_%": round(row["VIM_pct"], 4) if pd.notna(row["VIM_pct"]) else np.nan,
        "IV_calc_%": round(iv_calc*100.0, 4)
    })

out = pd.DataFrame(rows).reset_index(drop=True)
out


T = 44 jours ≈ 0.121 ans


,Strike,MidPrice,VIM_bbg_%,IV_calc_%
0,6685.0,171.95,14.0466,16.3546
1,6690.0,168.50,14.0033,16.2652
2,6695.0,165.20,13.9299,16.1895
3,6700.0,161.70,13.8644,16.0895
